# Platform Positioning

In [1]:
from __future__ import print_function

In [2]:
import random as random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.metrics import euclidean_distances

###  Generate Well Concept Data

In [3]:
from dummydatacreator import  GenerateWellConcept # This is not supported in Azure notebooks

In [4]:
num_of_concepts = 30

region_x_min = -60.62
region_x_max = -59.98
region_y_min = 9.87
region_y_max = 10.45
WellConcepts = [GenerateWellConcept(region_x_min, region_x_max, region_y_min, region_y_max) for i in range(num_of_concepts)]

In [5]:
WC_name, WC_points, WC_resource = list(zip(*WellConcepts))

In [6]:
print(WellConcepts[0])

['Dd-14', [-60.47991285152941, 10.041231255941554], 8.947266516840983]


### Well Concept / Platform Intersection

In [7]:
def ArePointsCloseTogether(pointA, pointB, distance):
    return (euclidean_distances([pointA], [pointB]) <= distance)[0][0]

print(ArePointsCloseTogether([10,2], [0,0], 10))

False


In [8]:
def AreWCsCloseToPlatforom(WellConceptsXY, PlatformXY, distance):    
    return [ArePointsCloseTogether(wcXY, PlatformXY, distance) for wcXY in WellConceptsXY]

In [115]:
def CalculateRankOfWCs(WC, plat, radii):
    '''Well Concepts are ranked according to binned distance from platform

    '''
    radii = np.sort(radii)
    num_radii = len(radii)

    #calculate distances of all well concepts from platform 
    distances = euclidean_distances(WC, [plat])
    distances = np.reshape(distances,[-1])

    #Reverse ranking - where 0 is closest to platform
    revranks = np.digitize(distances, radii, right = False) #i.e. includes left bound = 0

    # Ranking - where 0 is furthest from platform
    ranking = [num_radii - revrank for revrank in revranks]
    
    return ranking

In [134]:
def RankToColors(WC_rank):
    dct_colors = {0: None,
              1:"#FF8000",
              2:"#00CC00"}
    return dct_colors[WC_rank]

## Plotting

In [178]:
def plot_WC_Platform(WC_XY,WC_name, WC_res: np.array, plat_x, plat_y, plat_name, innerradius, outerradius, figsize):

    '''
    Plots Well Concepts and Platforms in Euclidean geometry
    '''
    
    
    #Unpack XY coordinates
    WC_x = [XY[0]for XY in WC_points] 
    WC_y = [XY[1]for XY in WC_points]
    

    fig, ax = plt.subplots(figsize=figsize)

    
    platform_radius = plt.Circle((plat_x, plat_y), radius = outerradius,fill=True,color='#FF8000', zorder=0 )
    ax.add_artist(platform_radius)
    
    platform_radius = plt.Circle((plat_x, plat_y), radius = innerradius,fill=True,color='#00CC00', zorder=1 )
    ax.add_artist(platform_radius)
    
    ax.scatter(plat_x, plat_y, marker = 'x', s=1000, zorder=2 )
   

    #Optional - no color desired as it was not clear
    #scatter = ax.scatter(WC_x, WC_y, c = WC_res, cmap="winter", s=100, label=WC_res, zorder=2)

    scatter = ax.scatter(WC_x, WC_y, c="#D3D3D3", s=100, label=WC_res, zorder=2)

        
    fig.colorbar(scatter, ax=ax)

    #Well concept labels
    for i, x in enumerate(WC_x):
        ax.annotate(WC_name[i], ([WC_x[i], WC_y[i]]), zorder=2)

    ax.annotate(plat_name, (plat_x, plat_y))
    plt.show()

In [179]:
def CreamMe(plat_x, plat_y, innerradius, outerradius, figsize):
    
    
    #Subset the WCs to only those which are on the creaming curve
    WC_rank = CalculateRankOfWCs(WC_points,[plat_x,plat_y],[0, innerradius,outerradius])

    #Construct dataframe of all well concepts
    df_WC = pd.DataFrame({"Well Concept": WC_name,"Resource": WC_resource, "Rank":WC_rank})
    
    #Reduce to df of well concepts on Creaming Curve (i.e. within the radii)
    df_WC_cc = df_WC[df_WC['Rank'] != 0].copy()
    
    
    df_WC_cc.loc[:,'Color'] = df_WC_cc['Rank'].map(lambda x: RankToColors(x))
    
    df_WC_cc.sort_values(by="Resource", inplace=True, ascending=False)
    
    
    

    fig, ax = plt.subplots(figsize=figsize)    

    ax.bar(df_WC_cc['Well Concept'], df_WC_cc['Resource'], color=df_WC_cc['Color'])

    ax.set_xlabel("Well Concept Name")
    ax.set_ylabel("Resource")
    plt.show()






In [180]:
userInputs = [widgets.BoundedFloatText(
    value=round(region_x_min +(random.random()*(region_x_max-region_x_min)), 2 ),
    min=region_x_min,
    max=region_x_max,
    step=0.01,
    description='X:',
    disabled=False
),
widgets.BoundedFloatText(
    value=round(region_y_min +(random.random()*(region_y_max-region_y_min)), 2 ),
    min=region_y_min,
    max=region_y_max,
    step=0.01,
    description='Y:',
    disabled=False
),
  widgets.BoundedFloatText(
    value=0.05,
    min=0.01,
    max=100,
    step=0.01,
    description='Radius:',
    disabled=False
)]



In [181]:
#%matplotlib inline 
from IPython.display import display

output = widgets.Output(layout={'border': '1px solid black'})


def on_button_clicked(event):
    output.clear_output()

    with output:
        plat_x = userInputs[0].value
        plat_y = userInputs[1].value
        radius = userInputs[2].value
        plot_WC_Platform(WC_points, WC_name, WC_resource,plat_x,plat_y,"platform", radius,radius*1.3, (12,10))
        CreamMe(plat_x, plat_y, radius,radius*1.3, (5,5))
    
        print("Button clicked.")        
        
button = widgets.Button(
    description='click me',
    layout={'width': '300px'},
    button_style='success', 
    tooltip='Run',
    icon='refresh'
)

display(widgets.Box(userInputs), button)


button.on_click(on_button_clicked)


Box(children=(BoundedFloatText(value=-60.2, description='X:', max=-59.98, min=-60.62, step=0.01), BoundedFloat…

Button(button_style='success', description='click me', icon='refresh', layout=Layout(width='300px'), style=But…

In [182]:
output

Output(layout=Layout(border='1px solid black'))